In [1]:
# Import necessary libraries [B]
%matplotlib inline
import time
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import matplotlib.dates as mdates
import numpy as np # Use numpy to convert to arrays
import pandas as pd # 引用套件並縮寫為 pd  
import pymysql
import pickle
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score
from datetime import datetime, timedelta
# Import necessary libraries [E]

In [27]:
# 參數定義 [B]
today_object = datetime.now()
today = today_object.strftime("%Y-%m-%d-%H-%M-%S")
today_midnight_object = datetime.strptime(today_object.strftime("%Y-%m-%d 00:00:00"), "%Y-%m-%d %H:%M:%S")
yesterday_object = today_midnight_object - timedelta(days=1)
yesterday = yesterday_object.strftime("%Y-%m-%d %H:%M:%S")
kmeans_pid_path = './../parameter-storage/pid/kmeans-pid.pid'
# 參數定義 [E]

In [5]:
# 讀取最後一個 kmeans 檔案的日期 [B]
with open(kmeans_pid_path) as f:
    kmeans_file_date = f.read()
    f.close()
print(kmeans_file_date)
# 讀取最後一個 kmeans 檔案的日期 [E]

2022-03-19-16-17-41


In [6]:
# 要儲存的新的 kmeans 檔案名稱 [B]
kmeans_modal_save_path = './../parameter-storage/kmeans/kmeans-modal-{}.pkl'.format(kmeans_file_date)
# 要儲存的新的 kmeans 檔案名稱 [E]

In [21]:
# 撈取需量資料 [B]
def getDemandData(begin_date, end_date): 
    try: 
        conn  =  pymysql.connect ( host = '　' ,  user = '　' ,  passwd = "　" ,  db = 'ncku_demand' ) 
        cur  =  conn.cursor() 
        select_sql = '''SELECT `id`, `demand_min`, `demand_quarter`, `Total_value`, `Temperature`, 
                        `T_Min`, `T_Max`, `day_of_year`, `data_week`, `data_hour`, 
                        `data_minute`, `data_weekday`, `minutes_of_the_day`, `data_date`, `datetime`
                        FROM `demand_with_weather_data` 
                        WHERE `datetime` > '{}' AND `datetime` <= '{}'
                        ORDER BY `datetime` '''.format(begin_date, end_date)
        result_object = cur.execute(select_sql)
        results_values_list = cur.fetchall()
        result_key_list = [i[0] for i in cur.description]
        print(select_sql)

        demand_dataframe = pd.DataFrame(results_values_list)
        demand_dataframe.columns = result_key_list
        timestamp = pd.to_datetime(demand_dataframe.datetime, infer_datetime_format=True).values.astype(float)
        demand_dataframe['timestamp'] = timestamp.tolist()
        demand_dataframe = demand_dataframe.set_index('datetime')
        cur.close () 
        conn.close()
        return demand_dataframe
    except Exception as e:
        print(e)
# 撈取需量資料 [E]

In [8]:
# 訓練模型並做儲存 [B]
def clusterTrainDemandData(n_clusters, demand_dataframe):
    X = demand_dataframe[['demand_quarter', 'minutes_of_the_day']]
    # 開始分類, 分離峰尖峰 [B]
    k_means_modal = KMeans(n_clusters).fit(X)
    # 儲存 k-means 模型
    pickle.dump(k_means_modal, open(kmeans_modal_save_path, "wb"))
    # 儲存檔案最後更新檔案日期 [B]
    kmeans_pid_write = open(kmeans_pid_path, 'w', encoding='utf-8')
    kmeans_pid_write.write(today)
    # 儲存檔案最後更新檔案日期 [E]
    return k_means_modal
# 訓練模型並做儲存 [E]

In [9]:
# 用已訓練好的模型做分類 [B]
def clusterDemandData(k_means_modal, demand_dataframe):
    X = demand_dataframe[['demand_quarter', 'minutes_of_the_day']]
    cluster_found = k_means_modal.predict(X)
    cluster_result = pd.Series(cluster_found, name='cluster')
    return cluster_result
# 用已訓練好的模型做分類 [E]

In [10]:
# 匯出分類結果於csv檔案 [B]
def exportClusterResult(demand_dataframe, cluster_result, title_name):
    demand_dataframe = demand_dataframe.assign(cluster=cluster_result.values) # assign(name=value)
    y_pred_pd = pd.DataFrame(data = demand_dataframe)
    y_pred_pd.to_csv('electricity-each-k-means-3-clusters-export-{}-20220319.csv'.format(title_name))
# 匯出分類結果於csv檔案 [E]

In [11]:
# 匯出分類結果於資料庫 [B]
def updateToDatabase(demand_dataframe, cluster_result):
    conn  =  pymysql.connect ( host = '　' ,  user = '　' ,  passwd = "　" ,  db = 'ncku_demand' ) 
    cur  =  conn.cursor() 
    demand_dataframe = demand_dataframe.assign(cluster=cluster_result.values) # assign(name=value)
    y_pred_pd = pd.DataFrame(data = demand_dataframe)
    for index, row in y_pred_pd.iterrows():
        update_sql = "UPDATE `demand_with_weather_data` SET `period_type`= {} WHERE `id`= {}".format( row['cluster'], row['id'])
        cur.execute(update_sql)
        conn.commit()
    cur.close () 
    conn.close()
# 匯出分類結果於資料庫 [E]

In [12]:
# 讀取 kmeans 模型 [B]
def loadKmeansModel(kmeans_modal_save_path):
    model = pickle.load(open(kmeans_modal_save_path, "rb"))
    return model
# 讀取 kmeans 模型 [E]

In [ ]:
# 讀取模型並預測 [B]
demand_dataframe = getDemandData(yesterday, today_midnight_object)
k_means_modal = loadKmeansModel(kmeans_modal_save_path)
cluster_result = clusterDemandData(k_means_modal, demand_dataframe)
updateToDatabase(demand_dataframe, cluster_result)
# 讀取模型並預測 [B]